In [35]:
import requests
import zipfile
from io import StringIO
import io
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
import numpy as np

In [7]:
mush_data_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data"
s =requests.get(mush_data_url).content
mush_data = pd.read_csv(io.StringIO(s.decode("utf-8")),header=None)

In [8]:
mush_data.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [39]:
columns = ["classes","cap-shape", "cap-surface", "cap-color", "bruises", "odor", "gill-attachment",
           "gill-spacing", "gill-size", "gill-color", "stalk-shape", "stalk-root", "stalk-surface-above-ring",
           "stalk-surface-below-ring", "stalk-color-above-ring", "stalk-color-below-ring", "veil-type", "veil-color",
           "ring-number", "ring-type", "spore-print-color", "population", "habitat"]
mush_data.columns = columns

In [13]:
mush_data.head()

,classes,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


## 将条目变成哑变量

In [17]:
# one hot encode 编码方式：https://blog.csdn.net/maymay_/article/details/80198468
mush_data_dummy = pd.get_dummies(mush_data[["gill-color","gill-attachment","odor","cap-color"]])

In [18]:
mush_data_dummy["flag"]=mush_data["classes"].map(lambda x: 1 if x=="p" else 0)

In [19]:
mush_data_dummy.head()

,gill-color_b,gill-color_e,gill-color_g,gill-color_h,gill-color_k,gill-color_n,gill-color_o,gill-color_p,gill-color_r,gill-color_u,...,cap-color_c,cap-color_e,cap-color_g,cap-color_n,cap-color_p,cap-color_r,cap-color_u,cap-color_w,cap-color_y,flag
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [21]:
X = mush_data_dummy.drop("flag",axis=1)
y = mush_data_dummy["flag"]
train_X,test_X,train_y,test_y=train_test_split(X,y,random_state=42)

In [22]:
model = DecisionTreeClassifier()
model.fit(train_X,train_y)
pred_y = model.predict(test_X)
model.score(test_X,test_y)

0.9935992122107337

## grid search to get best parameter

In [26]:
def decision_tree(trainX,trainY,testX,testY):
    param = {'criterion':['entropy'],'max_depth':list(range(7,12)),'min_samples_split':list(range(8,12))}
    grid = GridSearchCV(DecisionTreeClassifier(),param_grid=param,cv=5)
    grid.fit(trainX,trainY)
    print('best classifier',grid.best_params_,'best score',grid.best_score_)
    clf = DecisionTreeClassifier(**grid.best_params_)
    clf.fit(trainX, trainY)
    score = clf.score(testX, testY)
    print("test score is:", score)

In [27]:
decision_tree(train_X,train_y,test_X,test_y)

best classifier {'criterion': 'entropy', 'max_depth': 7, 'min_samples_split': 8} best score 0.9929427469553633
test score is: 0.9935992122107337


## bayesien algo

In [29]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [30]:
def naivebayes2(trainX, trainY, testX, testY):
    scaler = StandardScaler() # 归一化
    scaler.fit(trainX) 
    trainX2 = scaler.transform(trainX)
    testX2 = scaler.transform(testX)
    clf = GaussianNB().fit(trainX2,trainY)
    pred = clf.predict(testX2)
    accuracy=accuracy_score(pred,testY)
    return accuracy

In [31]:
naivebayes2(train_X,train_y,test_X,test_y)

0.9886755292959133

## 神经网络

In [ ]:
#神经网络
2.	def neural_network():
3.	    # start = time.time()
4.	    scaler = StandardScaler()#归一化处理
5.	    scaler.fit(trainX)
6.	    trainX1 = scaler.transform(trainX)
7.	    param = {'hidden_layer_sizes': [(20, 10), (10, 5)]}
8.	    grid = GridSearchCV(MLPClassifier(), param_grid=param, cv=5)
9.	    grid.fit(trainX1, trainY)
10.	    print('最优分类器:', grid.best_params_, '最优分数:', grid.best_score_)
11.	    # start2 = time.time()
12.	    testX1 = scaler.transform(testX)
13.	    print("Score of NN is", grid.score(testX1, testY))
14.	    return grid.best_score_


In [33]:
def neural_network(trainX, trainY, testX, testY):
    scaler = StandardScaler() # 归一化
    scaler.fit(trainX) 
    trainX1 = scaler.transform(trainX)
    testX1 = scaler.transform(testX)
    param = {'hidden_layer_sizes':[(20,10),(10,5)]}
    grid = GridSearchCV(MLPClassifier(),param_grid=param,cv=5)
    grid.fit(trainX1,trainY)
    print('best classifier:',grid.best_params_,'best parameters:',grid.best_score_)
    print('score of NN is:',grid.score(testX1,testY))
    return grid.best_score_

In [36]:
neural_network(train_X,train_y,test_X,test_y)

best classifier: {'hidden_layer_sizes': (20, 10)} best parameters: 0.9929427469553633
score of NN is: 0.9935992122107337


0.9929427469553633

In [37]:
def SVM(trainX,trainY,testX,testY):
    scaler = StandardScaler() # 归一化
    scaler.fit(trainX) 
    trainX1 = scaler.transform(trainX)
    testX1 = scaler.transform(testX)
    model = SVC(C=45,kernel='rbf',gamma=0.8)
    model.fit(trainX1,trainY)
    print("score of SVM:", model.score(testX1,testY))
    return model.score(testX1, testY)

In [38]:
SVM(train_X,train_y,test_X,test_y)

score of SVM: 0.9935992122107337


0.9935992122107337